In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker

sys.path.append('../../..')
sys.path.append('../../../../ajna_docs/commons')
from ajna_commons.flask.conf import SQL_URI

pd.options.display.float_format = '{:,.2f}'.format


In [2]:
engine = create_engine(SQL_URI)

In [3]:
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
SQL_PERDIMENTOS = '''
SELECT year(tg.create_date) as Ano, s.nome as Setor, count(distinct tg.id) as "Total de Autos", sum(tg.valor) as "Valor Total"
FROM ovr_ovrs o
inner join ovr_tgovr tg on tg.ovr_id = o.id
inner join ovr_setores s on s.id = o.setor_id
and year(tg.create_date) in (2024, 2025) and tg.valor is not null
group by year(tg.create_date), o.setor_id
'''
df_perdimentos = pd.read_sql(SQL_PERDIMENTOS, engine)


In [5]:
df_perdimentos['Valor médio'] = df_perdimentos['Valor Total'] / df_perdimentos['Total de Autos']

In [6]:
df_perdimentos

,Ano,Setor,Total de Autos,Valor Total,Valor médio
0,2024,ALFSTS/DIREP,3,"1,328,839.00","442,946.33"
1,2024,ALFSTS/DIREP/EQRIMP,65,"323,457,715.72","4,976,272.55"
2,2024,ALFSTS/SACIT/GMAB,180,"47,557,558.85","264,208.66"
3,2025,ALFSTS/DIREP/EQRIMP,55,"140,772,599.61","2,559,501.81"
4,2025,ALFSTS/DIREP/EQREXP,11,"4,945,606.10","449,600.55"
5,2025,ALFSTS/SACIT/GMAB,109,"15,138,865.62","138,888.68"


In [7]:
df = df_perdimentos.copy()
# Total geral por Ano
total_geral = df.groupby('Ano', as_index=False).agg({
    'Total de Autos': 'sum',
    'Valor Total': 'sum'
})
total_geral['Setor'] = 'Total Geral'

# Concatenar todos os resultados
df_final = pd.concat([df, total_geral], ignore_index=True)
# Ordenar para melhor visualização
df_final = df_final.sort_values(by=['Ano', 'Setor']).reset_index(drop=True)
df_final

,Ano,Setor,Total de Autos,Valor Total,Valor médio
0,2024,ALFSTS/DIREP,3,"1,328,839.00","442,946.33"
1,2024,ALFSTS/DIREP/EQRIMP,65,"323,457,715.72","4,976,272.55"
2,2024,ALFSTS/SACIT/GMAB,180,"47,557,558.85","264,208.66"
3,2024,Total Geral,248,"372,344,113.57",NaN
4,2025,ALFSTS/DIREP/EQREXP,11,"4,945,606.10","449,600.55"
5,2025,ALFSTS/DIREP/EQRIMP,55,"140,772,599.61","2,559,501.81"
6,2025,ALFSTS/SACIT/GMAB,109,"15,138,865.62","138,888.68"
7,2025,Total Geral,175,"160,857,071.33",NaN


# Sem GMAB

In [8]:
df = df_perdimentos[df_perdimentos.Setor != 'ALFSTS/SACIT/GMAB'].copy()
# Total geral por Ano
total_geral = df.groupby('Ano', as_index=False).agg({
    'Total de Autos': 'sum',
    'Valor Total': 'sum'
})
total_geral['Setor'] = 'Total Geral'

# Concatenar todos os resultados
df_final = pd.concat([df, total_geral], ignore_index=True)
# Ordenar para melhor visualização
df_final = df_final.sort_values(by=['Ano', 'Setor']).reset_index(drop=True)
df_final

,Ano,Setor,Total de Autos,Valor Total,Valor médio
0,2024,ALFSTS/DIREP,3,"1,328,839.00","442,946.33"
1,2024,ALFSTS/DIREP/EQRIMP,65,"323,457,715.72","4,976,272.55"
2,2024,Total Geral,68,"324,786,554.72",NaN
3,2025,ALFSTS/DIREP/EQREXP,11,"4,945,606.10","449,600.55"
4,2025,ALFSTS/DIREP/EQRIMP,55,"140,772,599.61","2,559,501.81"
5,2025,Total Geral,66,"145,718,205.71",NaN


In [14]:
SQL_RESULTADOS = '''
SELECT year(r.create_date) as Ano, e.TipoResultado, count(distinct o.id) as "Qtde", sum(r.valor) as "Valor Total"
FROM ovr_ovrs o
inner join ovr_resultados r on r.ovr_id = o.id
inner join Enumerado e on e.id = r.tipo_resultado
and year(r.create_date) in (2024, 2025) and r.valor is not null
group by year(r.create_date),  e.TipoResultado
'''
df_resultados = pd.read_sql(SQL_RESULTADOS, engine)

In [15]:
df_resultados

,Ano,TipoResultado,Qtde,Valor Total
0,2024,Apreensao,40,"2,385,283.82"
1,2024,Crédito,11,"13,930,501.48"
2,2024,Perdimento,248,"345,514,841.07"
3,2024,Sanção,4,0.00
4,2025,Apreensao,27,"14,919.69"
5,2025,Crédito,7,"1,776,781.00"
6,2025,Perdimento,172,"181,655,369.20"
7,2025,Sanção,1,1.00
